In [1]:
!pip install chardet


In [2]:
import ollama
import PyPDF2
import pandas as pd
import openpyxl
"""""
# Excel einlesen
df = pd.read_excel("dokument.xlsx")

# DataFrame zu Text konvertieren
content = df.to_string()

prompt=  f"Return keywords for a database search to find patents like {content}, no explanation, nothing else just give us some keywordsback  ."

response=ollama.generate(model="qwen3:8b", prompt=prompt)
list_response_key=response["response"]
print(list_response_key)

"""

import ollama
import pandas as pd
import openpyxl

# 1. Excel einlesen
df = pd.read_excel("dokument.xlsx")

# 2. Datenvalidierung: Pflichtfelder prüfen und unvollständige Zeilen entfernen
pflichtfelder = ["Abstract", "Title", "ApplicationDate"]  # anpassen je nach Struktur

zu_entfernende_indexe = []

for idx, zeile in df.iterrows():
    for feld in pflichtfelder:
        if feld not in df.columns or pd.isna(zeile.get(feld)) or str(zeile.get(feld)).strip() == "" or len(str(zeile.get(feld))) < 20:
            zu_entfernende_indexe.append(idx)
            break  # sobald ein Pflichtfeld fehlt, reicht das

df = df.drop(index=zu_entfernende_indexe)

print(f"{len(zu_entfernende_indexe)} ungültige Einträge entfernt.")
print(f"{len(df)} gültige Einträge nach Validierung.\n")

# 3. DataFrame zu String umwandeln
content = df.to_string(index=False)

# 4. Prompt an das Sprachmodell generieren
prompt = (
    f"Return keywords for a database search to find patents like the following content:\n\n"
    f"{content}\n\n"
    "Return only the keywords, no explanation, no formatting, just plain text keywords."
)

# 5. Anfrage an Ollama-Modell senden (z. B. qwen3:8b)
response = ollama.generate(model="qwen3:8b", prompt=prompt)

# 6. Ergebnis auslesen
list_response_key = response["response"]
print("Extrahierte Keywords:")
print(list_response_key)


ModuleNotFoundError: No module named 'ollama'

In [ ]:
import re
import ollama
import pandas as pd

def extract_keywords_from_ollama_response(text):
    """
    Extrahiert Keywords aus einer Ollama-Response und fügt OR zwischen jedem Wort ein.
    Speziell für Text mit zitierten technischen Begriffen.
    
    Args:
        text (str): Die Ollama-Response als String
    
    Returns:
        str: Keywords mit OR verknüpft
    """
    
    text = str(text).strip()
    keywords = []
    
    # Muster 1: Begriffe in Anführungszeichen extrahieren
    quoted_terms = re.findall(r'"([^"]+)"', text)
    if quoted_terms:
        keywords.extend(quoted_terms)
        print(f"Gefundene zitierte Begriffe: {len(quoted_terms)}")
    
    # Muster 2: Technische Begriffe nach "like" oder "terms like"
    after_like = re.findall(r'(?:terms?\s+like|like)\s+"([^"]+)"', text, re.IGNORECASE)
    keywords.extend(after_like)
    
    # Muster 3: Begriffe nach "include" 
    after_include = re.findall(r'include[s]?\s+"([^"]+)"', text, re.IGNORECASE)
    keywords.extend(after_include)
    
    # Muster 4: Zusätzliche technische Begriffe ohne Anführungszeichen
    # Suche nach bekannten technischen Patterns
    technical_patterns = [
        r'\bSn\s+alloy\b',
        r'\brelative\s+humidity\b', 
        r'\bratio\s+x/y\b',
        r'\btemperature\b',
        r'\bhumidity\b',
        r'\bnumerical\s+values\b'
    ]
    
    for pattern in technical_patterns:
        matches = re.findall(pattern, text, re.IGNORECASE)
        keywords.extend(matches)
    
    # Bereinige alle Keywords
    cleaned_keywords = []
    
    for keyword in keywords:
        if isinstance(keyword, str):
            # Grundbereinigung
            clean_keyword = keyword.strip()
            
            # Entferne führende Artikel
            clean_keyword = re.sub(r'^(the|a|an)\s+', '', clean_keyword, flags=re.IGNORECASE)
            
            # Teile zusammengesetzte Begriffe bei Kommas
            sub_keywords = [k.strip() for k in clean_keyword.split(',')]
            
            for sub_keyword in sub_keywords:
                sub_keyword = sub_keyword.strip()
                
                # Filtere sehr kurze oder unerwünschte Begriffe
                unwanted = ['and', 'or', 'the', 'specific', 'other', 'any', 'all']
                
                if (sub_keyword and 
                    len(sub_keyword) > 2 and 
                    sub_keyword.lower() not in unwanted):
                    cleaned_keywords.append(sub_keyword)
    
    # Entferne Duplikate, behalte Reihenfolge
    unique_keywords = list(dict.fromkeys(cleaned_keywords))
    
    # Füge OR zwischen allen Keywords ein
    result = " OR ".join(unique_keywords)
    
    return result

def main():
    try:
        # Excel einlesen
        df = pd.read_excel("dokument.xlsx")
        
        # DataFrame zu Text konvertieren
        content = df.to_string()
        
        # Prompt für Ollama
        prompt = f"Return keywords for a database search to find patents like {content}. No explanation, nothing else, just give us some keywords back."
        
        print("Sende Anfrage an Ollama...")
        
        # Ollama Response
        response=ollama.generate(model="qwen3:8b", prompt=prompt)
        list_response = response["response"]
        
        print("Ollama Response:")
        print(list_response)
        print("\n" + "="*50 + "\n")
        
        # Keywords extrahieren
        keywords_with_or = extract_keywords_from_ollama_response(list_response)
        
        print("Extrahierte Keywords mit OR:")
        print(keywords_with_or)
        
        print(f"\nAnzahl Keywords: {len(keywords_with_or.split(' OR ')) if keywords_with_or else 0}")
        
        # Speichere Ergebnis in Datei
        with open("extracted_keywords.txt", "w", encoding="utf-8") as f:
            f.write("Ollama Response:\n")
            f.write(list_response)
            f.write("\n\nExtracted Keywords:\n")
            f.write(keywords_with_or)
        
        print("\nErgebnis gespeichert in 'extracted_keywords.txt'")
        
        return keywords_with_or
        
    except FileNotFoundError:
        print("Fehler: 'dokument.xlsx' nicht gefunden!")
        return ""
    except Exception as e:
        print(f"Fehler: {e}")
        return ""

if __name__ == "__main__":
    main()

Sende Anfrage an Ollama...
Ollama Response:
<think>
Okay, let's tackle this. The user provided a long text about an electrochemical oxygen sensor and then a list of 20 claims. They want keywords related to the content. First, I need to understand the main points of the text.

The text describes an electrochemical oxygen sensor with components like a positive electrode, negative electrode, electrolyte solution, separation membrane, and a resistance element. It mentions specific parameters like current values, resistance, and conditions for the electrolyte solution, such as using a chelating agent like citric acid. There are also claims that specify details like the separation membrane's thickness, materials, and ratios involving the electrolyte volume and tin content.

Now, to extract keywords. The main components are the electrodes, electrolyte solution, separation membrane, and resistance element. The electrolyte solution's properties include chelating agents, citric acid, and specifi

In [ ]:
import requests
import json 
url = "https://www.searchapi.io/api/v1/search"
API_KEY="GAfKZo9ckfQ8xL8Y4H6qTZ1A"

params={
"enigne":"google_patents",
"q": "ChatGPT",
"api_key":"GAfKZo9ckfQ8xL8Y4H6qTZ1A",

}

response_api = requests.get(f"https://www.searchapi.io/api/v1/search?api_key={API_KEY}&=google_patents&q=ChatGPT")

print(response)
json_file= response.json()

with open("Test1.json","w") as file:
    json.dump(json_file, file, indent=4)

<Response [400]>


In [ ]:

datenbank="patent.csv"
datenbank_df = pd.read_csv(datenbank)
datenbank_content = datenbank_df.to_string()
print("CSV-Datei erfolgreich eingelesen")
        
# DataFrame zu Text konvertieren
content = df.to_string()

datenbank_response_csv="patentdb.csv"
datenbank_df = pd.read_csv(datenbank_response_csv)
datenbank_content = datenbank_df.to_string()
print("CSV-Datei erfolgreich eingelesen")

# Prompt für Ollama
prompt = f"Please provide feedback on whether these patents{datenbank_content} might conflict with other existing patents{datenbank_response_csv}. Return a list where 1 indicates a potential conflict and 0 indicates no conflict.Just Print the Final Answer as list other integeres tahn 0,1 are not valid"
        
print("Sende Anfrage an Ollama...")
response=ollama.generate(model="qwen3:8b", prompt=prompt)
list_response=response["response"]
print(list_response)


CSV-Datei erfolgreich eingelesen
CSV-Datei erfolgreich eingelesen
Sende Anfrage an Ollama...


2025-06-08 15:18:24,159 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


<think>
Okay, let's tackle this problem. The user has provided a list of patents and wants to know if there are any potential conflicts between them. The final answer should be a list where each entry is either 1 (conflict) or 0 (no conflict). Any other numbers are invalid.

First, I need to understand what constitutes a patent conflict. Typically, patent conflicts arise when two patents have overlapping claims, especially if they are in the same technical field and the claims are similar or the same. However, without specific details on the claims of each patent, I might need to rely on other indicators like the same inventors, same assignees, or similar classifications.

Looking at the data provided, each patent entry includes details like the patent number, country, status, assignee, inventors, and classification codes. Let me go through each entry and check for possible conflicts.

Starting with the first entry (index 0), it's a patent in the US (US9999999999A) with assignee "MAXEL

In [ ]:
import pandas as pd
import numpy as np
import os
import logging
from typing import Dict, List, Tuple, Optional
import chardet

# Logging konfigurieren
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class DataValidator:
    """Klasse für umfassende Datenvalidierung und Fehlerbehandlung"""
    
    def __init__(self):
        self.validation_results = {}
        
    def detect_encoding(self, file_path: str) -> str:
        """Erkennt die Kodierung einer Datei"""
        try:
            with open(file_path, 'rb') as file:
                raw_data = file.read()
                result = chardet.detect(raw_data)
                encoding = result['encoding']
                confidence = result['confidence']
                logger.info(f"Erkannte Kodierung: {encoding} (Konfidenz: {confidence:.2f})")
                return encoding
        except Exception as e:
            logger.error(f"Fehler bei Kodierungserkennung: {e}")
            return 'utf-8'
    
    def safe_read_csv(self, file_path: str, **kwargs) -> Optional[pd.DataFrame]:
        """Sicheres Einlesen von CSV-Dateien mit Fehlerbehandlung"""
        
        if not os.path.exists(file_path):
            logger.error(f"Datei nicht gefunden: {file_path}")
            return None
            
        # Standard-Parameter für robustes Einlesen
        default_params = {
            'encoding': None,
            'sep': None,  # Pandas erkennt automatisch
            'quotechar': '"',
            'escapechar': '\\',
            'error_bad_lines': False,
            'warn_bad_lines': True,
            'on_bad_lines': 'skip',  # Für neuere Pandas-Versionen
            'skipinitialspace': True,
            'skip_blank_lines': True
        }
        
        # Benutzerdefinierte Parameter überschreiben Standard-Parameter
        params = {**default_params, **kwargs}
        
        # Verschiedene Ansätze ausprobieren
        attempts = [
            # Versuch 1: Kodierung automatisch erkennen
            lambda: self._try_read_csv(file_path, **{**params, 'encoding': self.detect_encoding(file_path)}),
            
            # Versuch 2: Häufige Kodierungen
            lambda: self._try_read_csv(file_path, **{**params, 'encoding': 'utf-8'}),
            lambda: self._try_read_csv(file_path, **{**params, 'encoding': 'latin1'}),
            lambda: self._try_read_csv(file_path, **{**params, 'encoding': 'cp1252'}),
            
            # Versuch 3: Verschiedene Trennzeichen
            lambda: self._try_read_csv(file_path, **{**params, 'sep': ';', 'encoding': 'utf-8'}),
            lambda: self._try_read_csv(file_path, **{**params, 'sep': '\t', 'encoding': 'utf-8'}),
            
            # Versuch 4: Quote-Charaktere ignorieren
            lambda: self._try_read_csv(file_path, **{**params, 'quoting': 3, 'encoding': 'utf-8'}),  # QUOTE_NONE
            
            # Versuch 5: Als Text einlesen und manuell parsen
            lambda: self._manual_csv_parse(file_path)
        ]
        
        for i, attempt in enumerate(attempts, 1):
            try:
                df = attempt()
                if df is not None and not df.empty:
                    logger.info(f"CSV erfolgreich eingelesen (Versuch {i})")
                    logger.info(f"Shape: {df.shape}")
                    return df
            except Exception as e:
                logger.warning(f"Versuch {i} fehlgeschlagen: {e}")
                continue
        
        logger.error(f"Alle Versuche zum Einlesen von {file_path} fehlgeschlagen")
        return None
    
    def _try_read_csv(self, file_path: str, **kwargs) -> Optional[pd.DataFrame]:
        """Einzelner Versuch zum Einlesen einer CSV"""
        return pd.read_csv(file_path, **kwargs)
    
    def _manual_csv_parse(self, file_path: str) -> Optional[pd.DataFrame]:
        """Manuelles Parsen für schwierige CSV-Dateien"""
        try:
            with open(file_path, 'r', encoding=self.detect_encoding(file_path)) as file:
                lines = file.readlines()
                
            # Leere Zeilen entfernen
            lines = [line.strip() for line in lines if line.strip()]
            
            if not lines:
                return None
                
            # Trennzeichen bestimmen
            separators = [',', ';', '\t']
            best_sep = ','
            max_cols = 0
            
            for sep in separators:
                cols = len(lines[0].split(sep))
                if cols > max_cols:
                    max_cols = cols
                    best_sep = sep
            
            # Daten parsen
            data = []
            for line in lines:
                # Einfache Behandlung von Anführungszeichen
                row = []
                cells = line.split(best_sep)
                for cell in cells:
                    cell = cell.strip().strip('"').strip("'")
                    row.append(cell)
                data.append(row)
            
            # DataFrame erstellen
            if data:
                df = pd.DataFrame(data[1:], columns=data[0])
                logger.info("Manuelles Parsen erfolgreich")
                return df
                
        except Exception as e:
            logger.error(f"Manuelles Parsen fehlgeschlagen: {e}")
            
        return None
    
    def analyze_csv_structure(self, file_path: str) -> Dict:
        """Analysiert die Struktur der CSV-Datei vor dem Einlesen"""
        
        analysis = {
            'file_exists': os.path.exists(file_path),
            'file_size': 0,
            'encoding': None,
            'likely_separator': ',',
            'sample_lines': [],
            'potential_issues': []
        }
        
        if not analysis['file_exists']:
            analysis['potential_issues'].append("Datei existiert nicht")
            return analysis
            
        try:
            analysis['file_size'] = os.path.getsize(file_path)
            analysis['encoding'] = self.detect_encoding(file_path)
            
            # Erste Zeilen analysieren
            with open(file_path, 'r', encoding=analysis['encoding']) as file:
                sample_lines = [file.readline().strip() for _ in range(5)]
                analysis['sample_lines'] = [line for line in sample_lines if line]
            
            # Trennzeichen bestimmen
            if analysis['sample_lines']:
                separators = [',', ';', '\t', '|']
                sep_counts = {}
                for sep in separators:
                    sep_counts[sep] = analysis['sample_lines'][0].count(sep)
                
                analysis['likely_separator'] = max(sep_counts, key=sep_counts.get)
                
                # Potenzielle Probleme erkennen
                first_line = analysis['sample_lines'][0]
                if '"' in first_line and first_line.count('"') % 2 != 0:
                    analysis['potential_issues'].append("Unvollständige Anführungszeichen")
                
                if any(line.endswith(',') or line.endswith(';') for line in analysis['sample_lines']):
                    analysis['potential_issues'].append("Zeilen enden mit Trennzeichen")
                    
        except Exception as e:
            analysis['potential_issues'].append(f"Analysefehler: {e}")
            
        return analysis
    
    def validate_dataframe(self, df: pd.DataFrame, name: str = "DataFrame") -> Dict:
        """Umfassende Validierung eines DataFrames"""
        
        validation_results = {
            'name': name,
            'shape': df.shape,
            'columns': list(df.columns),
            'dtypes': df.dtypes.to_dict(),
            'missing_values': df.isnull().sum().to_dict(),
            'duplicate_rows': df.duplicated().sum(),
            'memory_usage': df.memory_usage(deep=True).sum(),
            'issues': []
        }
        
        # Fehlende Werte prüfen
        missing_percentage = (df.isnull().sum() / len(df) * 100)
        high_missing = missing_percentage[missing_percentage > 50]
        if not high_missing.empty:
            validation_results['issues'].append(
                f"Spalten mit >50% fehlenden Werten: {high_missing.to_dict()}"
            )
        
        # Datentypen prüfen
        object_columns = df.select_dtypes(include=['object']).columns
        if len(object_columns) > 0:
            validation_results['object_columns'] = list(object_columns)
            
        # Duplikate prüfen
        if validation_results['duplicate_rows'] > 0:
            validation_results['issues'].append(
                f"{validation_results['duplicate_rows']} doppelte Zeilen gefunden"
            )
        
        # Spalten mit nur einem Wert
        single_value_cols = [col for col in df.columns if df[col].nunique() <= 1]
        if single_value_cols:
            validation_results['single_value_columns'] = single_value_cols
            validation_results['issues'].append(
                f"Spalten mit nur einem Wert: {single_value_cols}"
            )
        
        self.validation_results[name] = validation_results
        logger.info(f"Validierung für {name} abgeschlossen")
        
        return validation_results
    
    def clean_dataframe(self, df: pd.DataFrame) -> pd.DataFrame:
        """Grundlegende Datenbereinigung"""
        
        df_cleaned = df.copy()
        
        # Duplikate entfernen
        df_cleaned = df_cleaned.drop_duplicates()
        
        # Spalten mit nur NaN-Werten entfernen
        df_cleaned = df_cleaned.dropna(axis=1, how='all')
        
        # Zeilen mit nur NaN-Werten entfernen
        df_cleaned = df_cleaned.dropna(axis=0, how='all')
        
        # Spaltennamen bereinigen
        df_cleaned.columns = df_cleaned.columns.str.strip()
        
        logger.info(f"Datenbereinigung abgeschlossen. Shape: {df.shape} -> {df_cleaned.shape}")
        
        return df_cleaned
    
    def generate_report(self) -> str:
        """Generiert einen Validierungsbericht"""
        
        report = "\n" + "="*50 + "\n"
        report += "DATENVALIDIERUNGSBERICHT\n"
        report += "="*50 + "\n"
        
        for name, results in self.validation_results.items():
            report += f"\n--- {results['name']} ---\n"
            report += f"Shape: {results['shape']}\n"
            report += f"Spalten: {len(results['columns'])}\n"
            report += f"Doppelte Zeilen: {results['duplicate_rows']}\n"
            report += f"Speicherverbrauch: {results['memory_usage'] / 1024:.2f} KB\n"
            
            if results['issues']:
                report += "\nProbleme gefunden:\n"
                for issue in results['issues']:
                    report += f"  - {issue}\n"
            else:
                report += "\nKeine kritischen Probleme gefunden.\n"
        
        return report

# Hauptfunktion für CSV-Validierung
def validate_patent_csv():
    """Validiert nur die CSV-Datei mit robuster Fehlerbehandlung"""
    
    validator = DataValidator()
    
    # CSV-Datei einlesen
    print("Versuche CSV-Datei einzulesen...")
    csv_df = validator.safe_read_csv("patentdb.csv")
    
    if csv_df is not None:
        print("CSV-Datei erfolgreich eingelesen")
        print(f"Ursprüngliche Shape: {csv_df.shape}")
        
        # Daten bereinigen
        csv_df_cleaned = validator.clean_dataframe(csv_df)
        print(f"Shape nach Bereinigung: {csv_df_cleaned.shape}")
        
        # Validierung durchführen
        validator.validate_dataframe(csv_df_cleaned, "Patent-Datenbank")
        
        # Validierungsbericht ausgeben
        print(validator.generate_report())
        
        return csv_df_cleaned
    else:
        print("FEHLER: CSV-Datei konnte nicht eingelesen werden")
        print("Mögliche Ursachen:")
        print("- Datei 'patentdb.csv' existiert nicht")
        print("- Kodierungsprobleme")
        print("- Beschädigte oder unvollständige Daten")
        print("- Problematische Anführungszeichen oder Trennzeichen")
        return None

# Verwendungsbeispiel
if __name__ == "__main__":
    # Nur CSV validieren
    csv_df = validate_patent_csv()
    
    if csv_df is not None:
        print("\n" + "="*50)
        print("CSV-VALIDIERUNG ERFOLGREICH")
        print("="*50)
        print(f"Finale Datenbank Shape: {csv_df.shape}")
        print(f"Spalten: {list(csv_df.columns)}")
        
        # Excel separat einlesen (ohne Validierung)
        try:
            excel_df = pd.read_excel("dokument.xlsx")
            content = excel_df.to_string()
            print("Excel-Datei erfolgreich eingelesen")
            
            # Datenbank-Content für Prompt vorbereiten
            datenbank_content = csv_df.to_string()
            
            prompt = f"""Please provide feedback on whether these patents:
{content}

might conflict with other existing patents:
{datenbank_content}

Return a list where 1 indicates a potential conflict and 0 indicates no conflict.
Just Print the Final Answer"""
            
            print("\nDaten für Ollama vorbereitet...")
            print(f"Patent-Dokument Shape: {excel_df.shape}")
            print(f"Validierte Datenbank Shape: {csv_df.shape}")
            
            # Hier Ollama-Aufruf einfügen:
            # response = ollama.generate(model="qwen3:8b", prompt=prompt)
            # list_response = response["response"]
            # print(list_response)
            
        except Exception as e:
            print(f"Fehler beim Einlesen der Excel-Datei: {e}")
            print("Nur CSV-Validierung durchgeführt")
            
    else:
        print("\n" + "="*50)
        print("CSV-VALIDIERUNG FEHLGESCHLAGEN")
        print("="*50)
        print("Bitte überprüfen Sie die Datei 'patentdb.csv'")

2025-06-08 14:48:04,035 - INFO - Erkannte Kodierung: UTF-8-SIG (Konfidenz: 1.00)
2025-06-08 14:48:04,035 - WARNING - Versuch 1 fehlgeschlagen: read_csv() got an unexpected keyword argument 'error_bad_lines'. Did you mean 'on_bad_lines'?
2025-06-08 14:48:04,036 - WARNING - Versuch 2 fehlgeschlagen: read_csv() got an unexpected keyword argument 'error_bad_lines'. Did you mean 'on_bad_lines'?
2025-06-08 14:48:04,036 - WARNING - Versuch 3 fehlgeschlagen: read_csv() got an unexpected keyword argument 'error_bad_lines'. Did you mean 'on_bad_lines'?
2025-06-08 14:48:04,037 - WARNING - Versuch 4 fehlgeschlagen: read_csv() got an unexpected keyword argument 'error_bad_lines'. Did you mean 'on_bad_lines'?
2025-06-08 14:48:04,037 - WARNING - Versuch 5 fehlgeschlagen: read_csv() got an unexpected keyword argument 'error_bad_lines'. Did you mean 'on_bad_lines'?
2025-06-08 14:48:04,037 - WARNING - Versuch 6 fehlgeschlagen: read_csv() got an unexpected keyword argument 'error_bad_lines'. Did you mean

Versuche CSV-Datei einzulesen...
FEHLER: CSV-Datei konnte nicht eingelesen werden
Mögliche Ursachen:
- Datei 'patentdb.csv' existiert nicht
- Kodierungsprobleme
- Beschädigte oder unvollständige Daten
- Problematische Anführungszeichen oder Trennzeichen

CSV-VALIDIERUNG FEHLGESCHLAGEN
Bitte überprüfen Sie die Datei 'patentdb.csv'
